# Volcanoes in US

Code example: https://pythonhow.com/web-mapping-with-python-and-folium/

Data volcanoes: https://pythonhow.com/data/Volcanoes_USA.txt

Data polygons for the population: https://pythonhow.com/data/world_geojson_from_ogr.json

In [1]:
import numpy as np
import pandas as pd
import folium
import logging

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [3]:
INPUT_FOLDER_NAME = "/Users/abuzatu/Work/data/geo_spatial/USA/volcanos"
input_file_name_volcanoes = f"{INPUT_FOLDER_NAME}/usa_volcanoes.txt"
input_file_name_population = f"{INPUT_FOLDER_NAME}/world_geojson_from_ogr.json"

In [4]:
df = pd.read_csv(input_file_name_volcanoes)
df

,VOLCANX020,NUMBER,NAME,LOCATION,STATUS,ELEV,TYPE,TIMEFRAME,LAT,LON
0,509.0,1201-01=,Baker,US-Washington,Historical,3285.0,Stratovolcanoes,D3,48.776798,-121.810997
1,511.0,1201-02-,Glacier Peak,US-Washington,Tephrochronology,3213.0,Stratovolcano,D4,48.111801,-121.111000
2,513.0,1201-03-,Rainier,US-Washington,Dendrochronology,4392.0,Stratovolcano,D3,46.869801,-121.751000
3,515.0,1201-05-,St. Helens,US-Washington,Historical,2549.0,Stratovolcano,D1,46.199799,-122.181000
4,516.0,1201-04-,Adams,US-Washington,Tephrochronology,3742.0,Stratovolcano,D6,46.205799,-121.490997
...,...,...,...,...,...,...,...,...,...,...
57,580.0,1210-03-,Valles Caldera,US-New Mexico,Pleistocene-Fumarolic,3430.0,Caldera,Q,35.869999,-106.570999
58,581.0,1209-02-,Sunset Crater,US-Arizona,Dendrochronology,2447.0,Cinder cone,D6,35.369999,-111.501000
59,584.0,1203-20-,Amboy,US-California,Holocene,288.0,Cinder cone,U,34.549999,-115.780998
60,586.0,1210-02-,Zuni-Bandera,US-New Mexico,Anthropology,2550.0,Volcanic field,D7,34.799999,-108.001000


Let's color the volcanos by grouping them by their their elevation

In [5]:
def set_color(df, nb_step):
    list_color = ["lightred", "darkgreen", "cadetblue", "orange", "gray", "pink", "white", "blue", "lightgray", "lightblue", "black", "darkblue", "purple", "green", "lightgreen", "beige", "darkpurple", "red", "darkred"]
    if nb_step > len(list_color)+1:
        raise RuntimeError(f"nb_step={nb_step} < len(list_color)={len(list_color)}")
    vmin = min(df['ELEV'])
    vmax = max(df['ELEV'])
    logging.debug(f"vmin={vmin}, vmax={vmax}, nb_step={nb_step}")
    step_size = np.ceil((vmax - vmin) / nb_step)
    def get_color(elevation):
        color = None
        for i in range(0, nb_step+1):
            low  = vmin + i * step_size
            high = vmin + (i+1) * step_size
            logging.debug(f"i={i}, low={low}, high={high}, step_size={step_size}")
            if low <= elevation < high:
                color = list_color[i]
                break
        logging.debug(f"elevation={elevation}, color={color}")
        return color
    df["color"] = df.ELEV.map(lambda x: get_color(x))
    return df

In [6]:
nb_step = 5
set_color(df, nb_step)
df

,VOLCANX020,NUMBER,NAME,LOCATION,STATUS,ELEV,TYPE,TIMEFRAME,LAT,LON,color
0,509.0,1201-01=,Baker,US-Washington,Historical,3285.0,Stratovolcanoes,D3,48.776798,-121.810997,orange
1,511.0,1201-02-,Glacier Peak,US-Washington,Tephrochronology,3213.0,Stratovolcano,D4,48.111801,-121.111000,orange
2,513.0,1201-03-,Rainier,US-Washington,Dendrochronology,4392.0,Stratovolcano,D3,46.869801,-121.751000,gray
3,515.0,1201-05-,St. Helens,US-Washington,Historical,2549.0,Stratovolcano,D1,46.199799,-122.181000,cadetblue
4,516.0,1201-04-,Adams,US-Washington,Tephrochronology,3742.0,Stratovolcano,D6,46.205799,-121.490997,gray
...,...,...,...,...,...,...,...,...,...,...,...
57,580.0,1210-03-,Valles Caldera,US-New Mexico,Pleistocene-Fumarolic,3430.0,Caldera,Q,35.869999,-106.570999,orange
58,581.0,1209-02-,Sunset Crater,US-Arizona,Dendrochronology,2447.0,Cinder cone,D6,35.369999,-111.501000,cadetblue
59,584.0,1203-20-,Amboy,US-California,Holocene,288.0,Cinder cone,U,34.549999,-115.780998,lightred
60,586.0,1210-02-,Zuni-Bandera,US-New Mexico,Anthropology,2550.0,Volcanic field,D7,34.799999,-108.001000,cadetblue


# Plot the volcanoes on a folium map

In [7]:
# create map
folium_map = folium.Map(location = [df['LAT'].mean(), df['LON'].mean()], zoom_start = 4)
if True:
    # add the National Geography Map appropriate for visualizing volcanoes
    name = "Nat Geo Map"
    url = "https://server.arcgisonline.com/ArcGIS/rest/services/NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}"
    attr = "Tiles &copy; Esri &mdash; National Geographic, Esri, DeLorme, NAVTEQ, UNEP-WCMC, USGS, NASA, ESA, METI, NRCAN, GEBCO, NOAA, iPC"
    folium.TileLayer(url, attr = attr, name = name).add_to(folium_map)
if True:
    folium.TileLayer(name = "Stamen Terrain").add_to(folium_map)
    
# add a marker for each volcano by adding to a feature group
fg = folium.FeatureGroup(name = "Volcano Locations")
for lat, lon, name, elevation, color in zip(df["LAT"], df["LON"], df["NAME"], df["ELEV"], df["color"]):
    folium.Marker(
            location = [lat, lon],
            icon = folium.Icon(color = color, icon_color = "green"),
            tooltip = f"Volcano {name} of elevation {elevation}",
            popup = f"Volcano {name} of elevation {elevation}",
        ).add_to(fg)
fg.add_to(folium_map) # the equivalent command folium_map.add_child(fg)

# add the layer control bottom
folium_map.add_child(folium.LayerControl())

# plot the map
folium_map

In [8]:
folium_map.save(outfile = "folium_map_volcanoes.html")